In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from ticker_download_predict_upload import DownloadPredictUpload

In [2]:
dpu = DownloadPredictUpload()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
tickers = ["I:SPX", "QQQ", "NVDA", "WMT", "TGT"]

In [4]:
long_df_filename = os.path.join("input", f"Year of Tickers {dpu.get_today_date()}.csv")
if os.path.exists(long_df_filename):
    long_df = pd.read_csv(long_df_filename)
    long_df["datetime"] = pd.to_datetime(long_df["datetime"])
    long_df["datetime"] = long_df["datetime"].apply(lambda x: pd.Timestamp(x).replace(hour=23, minute=59, second=59))
    long_df.set_index("datetime", inplace=True)
    long_df.sort_index(inplace=True)
else:
    date_from = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 253)
    date_to = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 1).replace(
        hour=23, minute=59, second=59
    )
    print(date_from, date_to)
    long_df = dpu.get_tickers(tickers, date_from=date_from, date_to=date_to)
    long_df.to_csv(long_df_filename, index=True)
long_df

,ticker,open,high,low,close,volume,vwap,transactions
datetime,,,,,,,,
2024-03-25 23:59:59,NVDA,93.941,96.766,93.5100,95.002,552047330.0,95.4480,1113327.0
2024-03-25 23:59:59,WMT,60.870,60.985,60.3600,60.570,14186627.0,60.5604,105344.0
2024-03-25 23:59:59,QQQ,443.550,446.260,442.5400,444.760,27779904.0,445.0143,304143.0
2024-03-25 23:59:59,TGT,170.000,173.580,169.7300,172.660,4262579.0,172.3819,59716.0
2024-03-25 23:59:59,I:SPX,5219.520,5229.090,5216.0900,5218.190,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-03-28 23:59:59,WMT,85.430,85.830,84.7000,85.150,14789418.0,85.1368,172741.0
2025-03-28 23:59:59,QQQ,479.810,480.520,468.0500,468.940,46399302.0,472.5380,501401.0
2025-03-28 23:59:59,NVDA,111.485,112.870,109.0701,109.670,229862293.0,110.1686,1847381.0


In [6]:
wide_df = dpu.pivot_ticker_close_wide(long_df)
wide_df

ticker,I:SPX,NVDA,QQQ,TGT,WMT
2024-03-25 17:00:00,5218.19,95.002,444.76,172.66,60.57
2024-03-26 17:00:00,5203.58,92.561,443.32,172.64,60.51
2024-03-27 17:00:00,5248.49,90.250,444.83,174.67,60.72
2024-03-28 17:00:00,5254.35,90.356,444.01,177.21,60.17
2024-04-01 17:00:00,5243.77,90.363,444.95,177.82,60.00
...,...,...,...,...,...
2025-03-24 17:00:00,5767.57,121.410,490.66,108.35,87.49
2025-03-25 17:00:00,5776.65,120.690,493.46,105.40,84.76
2025-03-26 17:00:00,5712.20,113.760,484.38,106.09,85.21
2025-03-27 17:00:00,5693.31,111.430,481.62,106.51,85.63


In [7]:
wide_df.isna().sum().sum()

np.int64(0)

In [8]:
returns_df = wide_df.pct_change()
returns_df = returns_df.iloc[1:] * 100
returns_df

ticker,I:SPX,NVDA,QQQ,TGT,WMT
2024-03-26 17:00:00,-0.279982,-2.569420,-0.323770,-0.011583,-0.099059
2024-03-27 17:00:00,0.863060,-2.496732,0.340612,1.175857,0.347050
2024-03-28 17:00:00,0.111651,0.117452,-0.184340,1.454171,-0.905797
2024-04-01 17:00:00,-0.201357,0.007747,0.211707,0.344224,-0.282533
2024-04-02 17:00:00,-0.723907,-1.008156,-0.863018,-0.888539,-1.400000
...,...,...,...,...,...
2025-03-24 17:00:00,1.764604,3.152082,2.042259,4.122622,1.756222
2025-03-25 17:00:00,0.157432,-0.593032,0.570660,-2.722658,-3.120357
2025-03-26 17:00:00,-1.115699,-5.741984,-1.840068,0.654649,0.530911
2025-03-27 17:00:00,-0.330696,-2.048172,-0.569801,0.395890,0.492900
